In [1]:
import pandas as pd
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
import importlib

package_name = "mlflow"
try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ModuleNotFoundError:
    print(f"{package_name} not found. Installing...")
    %pip install {package_name}


mlflow is already installed.


In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-exp-2")

2023/04/04 05:13:27 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/04 05:13:27 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week2/mlruns/2', creation_time=1679093170978, experiment_id='2', last_update_time=1679093170978, lifecycle_stage='active', name='nyc-taxi-exp-2', tags={}>

In [14]:
lasso = Lasso(alpha=0.01)

In [15]:
lasso.fit(X_train,y_train)

Lasso(alpha=0.01)

In [16]:
import mlflow
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from joblib import dump, load
import tempfile
import os

models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

with mlflow.start_run(description="Adding Artifact as test"):

    mlflow.set_tag("developer", "bt")
    mlflow.set_tag("model", "Lasso")
    mlflow.set_tag("date", "2023-04-04")

    mlflow.log_param("train data", "/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet")
    mlflow.log_param("test data", "/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet")

    alpha = 0.1

    mlflow.log_param("alpha", alpha)

    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)

    model_path = os.path.join(models_dir, "lasso_reg.bin")

    dump(lasso, model_path)

    mlflow.log_artifact(local_path=model_path, artifact_path="models_pickle")

    # # Save the model to a temporary file
    # with tempfile.NamedTemporaryFile(suffix=".bin", delete=False) as temp_file:
    #     dump(lasso, temp_file.name)
    #     temp_file.flush()

    #     # Log the temporary model file as an artifact
    #     mlflow.log_artifact(local_path=temp_file.name, artifact_path="models_pickle")

    # # Remove the temporary file after logging the artifact
    # os.remove(temp_file.name)


In [13]:
with mlflow.start_run(description="Adding Artifact as test"):

    mlflow.set_tag("developer","bt")
    mlflow.set_tag("model","Lasso")
    mlflow.set_tag("date","2023-04-04")

    mlflow.log_param("train data","/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet")
    mlflow.log_param("test data","/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet")

    alpha = 0.1

    mlflow.log_param("alpha",alpha)

    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train,y_train)

    y_pred = lasso.predict(X_val)

    rmse = mean_squared_error(y_val,y_pred,squared=False)

    mlflow.log_metric("rmse",rmse)
    mlflow.log_artifact(artifact_path='models_pickle')


TypeError: log_artifact() missing 1 required positional argument: 'local_path'

In [11]:
import os

In [12]:
os.getcwd()

'/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week2'

In [11]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val,label=y_val)

Define Objective Function

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model','xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val,y_pred,squared=False)
        mlflow.log_metric("rmse",rmse)
        mlflow.log_artifact(local_path='models/lin_reg.bin',artifact_path='models_pickle/')
    return {'loss': rmse, 'status': STATUS_OK}

Define Search Space

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate' : hp.loguniform('learning_rate',-3,0),
    'reg_alpha' : hp.loguniform('reg_alpha',-5,-1),
    'reg_lambda' : hp.loguniform('reg_lambda',-6,-1),
    'min_child_weight': hp.loguniform('min_child_weight',-1,3),
    'objective':'reg:linear',
    'seed': 43
}

In [24]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

ERROR [hyperopt.fmin] job exception: name 'train' is not defined


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


NameError: name 'train' is not defined